# Workload rtapp + Idle States Residency Analysis

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup(level=100)

# Execute this cell to enabled devlib debugging statements
logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
%matplotlib inline

import os

# Support to access the remote target
from env import TestEnv

# Support to access cpuidle information from the target
from devlib import *

# Support to configure and run RTApp based workloads
from wlgen import RTA, Ramp, Periodic

# Support for trace events analysis
from trace import Trace
from trace import ResidencyTime, ResidencyData

# DataFrame support
import pandas as pd
from pandas import DataFrame

# Trappy (plots) support
from trappy import ILinePlot
from trappy.stats.grammar import Parser

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

# CPUIdle break residency statistics

In [3]:
def dfCPUIdleStateEstimationStatistics(ia, cpu, residency):
    
    #residency = residency / 1000000
    
    idle_df = ia._dfg_trace_event('cpu_idle')
    cpu_idle = idle_df[idle_df.cpu_id == cpu]

    # Generate valid idle states
    available_idles = sorted(idle_df.state.unique())
    available_idles = available_idles[1:]

    cpu_is_idle = ia._trace.getCPUActiveSignal(cpu) ^ 1
    cpu_idle = cpu_idle.join(cpu_is_idle.to_frame(name='is_idle'),
                                 how='outer')
    cpu_idle.fillna(method='ffill', inplace=True)

    idle_time = pd.DataFrame({'over_est' : 0, 'under_est' : 0, 'good_est' : 0, 'cpu' : 0}, index=available_idles)

    for i in available_idles:
        idle_state = cpu_idle.state.apply(
            lambda x: 1 if x == i else 0
        )
        #print idle_state
        idle_t = cpu_idle.is_idle * idle_state
        #print idle_t
        # Compute total time by integrating the square wave
        #print ia._trace.integrate_square_wave(idle_t)
        idle_t.iloc[-1] = 0.0

        # Compact signal to obtain only 1-0-1-0 sequences
        comp_sig = idle_t.loc[idle_t.shift() != idle_t]
        #print comp_sig
        if comp_sig.iloc[0] == 0.0:
            idle_interval = (comp_sig.iloc[2::2].index - comp_sig.iloc[1:-1:2].index)
        else:
            idle_interval = (comp_sig.iloc[1::2].index - comp_sig.iloc[:-1:2].index)
        
        idle_interval *= 1000000
        
        idle_time.at[i, 'cpu']  = cpu
        idle_time.at[i, 'over_est']  = len(idle_interval[idle_interval < residency[i]])
        idle_time.at[i, 'under_est'] = len(idle_interval[idle_interval > residency[i+1]])
        idle_time.at[i, 'good_est']  = len(idle_interval[(idle_interval >= residency[i]) & (idle_interval < residency[i+1])])          
        
    return idle_time

In [4]:
from collections import namedtuple

ResidencyTimes = namedtuple('ResidencyTimes', ['label', 'stats'])

def dfAllCPUIdleStateEstimationStatistics(ia, residency, cpus=None):

    if not ia._trace.hasEvents('cpu_idle'):
        ia._log.warning('Events [cpu_idle] not found, '
                        'plot DISABLED!')
        return

    if cpus is None:
        # Generate plots only for available CPUs
        cpuidle_data = ia._dfg_trace_event('cpu_idle')
        _cpus = range(cpuidle_data.cpu_id.max() + 1)
    else:
        _cpus = listify(cpus)

    # Generate valid idle states
    idle_df = ia._dfg_trace_event('cpu_idle')
    available_idles = sorted(idle_df.state.unique())
    available_idles = available_idles[1:]
    
    residencies = pd.DataFrame()
    total_r = pd.DataFrame({'over_est' : 0, 'under_est' : 0, 'good_est' : 0, 'cpu' : -1}, index=available_idles)
    for cpu in _cpus:
        r = dfCPUIdleStateEstimationStatistics(ia, cpu, residency)
        residencies = pd.concat([residencies, r])
        total_r['over_est']  = total_r['over_est']  + r['over_est']
        total_r['under_est'] = total_r['under_est'] + r['under_est']
        total_r['good_est']  = total_r['good_est']  + r['good_est']
        
    residencies = pd.concat([residencies, total_r])
    return residencies

# Hikey620 Platform Configurations

In [5]:
# Setup target configuration
hikey620_conf = {
    "platform"     : 'linux',
    "board"        : 'hikey',
    "host"         : "192.168.1.20",
    "username"     : "root",
    "password"     : "root",
    "results_dir"  : "test",
    "modules"      : [ 'cpufreq', 'cpuidle' ],
    "ftrace"       : {
        "events" : [ "cpu_idle", "cpu_capacity", "cpu_frequency", "sched_switch", "sched_load_avg_cpu" ],
        "buffsize" : 50 * 1024,
    },
    "tools"        : [ 'trace-cmd', 'taskset', 'rt-app', 'sysbench' ],
    "rtapp-calib"  : {
        "0": 465, "1": 465, "2": 465, "3": 465, "4": 465, "5": 465, "6": 465, "7": 465,
    },    
}

In [6]:
hikey620_platform = {
    'kernel': {
        'major': 4,
        'sha1': '78e8b90',
        'version': '57 SMP PREEMPT Sat Jul 1 15:42:00 CST 2017',
        'version_number': 4,
        'parts': [4, 4, 74],
        'rc': None,
        'release': '4.4.74-g78e8b90',
        'minor': 74
    },
    
    'freqs': {
        'big':    [ 280000, 432000, 729000, 960000, 1200000 ],
        'little': [ 280000, 432000, 729000, 960000, 1200000 ]
    },
    
    'nrg_model': {
        'big': {
            'cluster': {
                'nrg_max': 112
            },
            'cpu': {
                'cap_max': 1024,
                'nrg_max': 607
            }
        },
        'little': {
            'cluster': {
                'nrg_max': 112
            },
            'cpu': {
                'cap_max': 1024,
                'nrg_max': 607
            }
        }
    },
    
    'clusters': {
        'big': [4, 5, 6, 7],
        'little': [0, 1, 2, 3]
    },
    'cpus_count': 8,
    'topology': [[0, 1, 2, 3], [4, 5, 6, 7]]
}

In [7]:
# Support to access the remote target
from env import TestEnv

te = TestEnv(hikey620_conf)
target = te.target

2018-07-23 15:15:27,737 INFO    : TestEnv      : Using base path: /home/leoy/Work2/Develop/tools/lisa
2018-07-23 15:15:27,756 INFO    : TestEnv      : Loading custom (inline) target configuration
2018-07-23 15:15:27,758 WARNING : TestEnv      : Wipe previous contents of the results folder:
2018-07-23 15:15:27,760 WARNING : TestEnv      :    /home/leoy/Work2/Develop/tools/lisa/results/test
2018-07-23 15:15:27,771 INFO    : TestEnv      : Devlib modules to load: ['cpuidle', 'cpufreq']
2018-07-23 15:15:27,773 INFO    : TestEnv      : Connecting linux target:
2018-07-23 15:15:27,774 INFO    : TestEnv      :   username : root
2018-07-23 15:15:27,776 INFO    : TestEnv      :       host : 192.168.1.20
2018-07-23 15:15:27,777 INFO    : TestEnv      :   password : root
2018-07-23 15:15:27,779 INFO    : TestEnv      : Connection settings:
2018-07-23 15:15:27,781 INFO    : TestEnv      :    {'username': 'root', 'host': '192.168.1.20', 'password': 'root'}
2018-07-23 15:15:27,791 DEBUG   : ssh     

# Workload execution utility

In [8]:
def execute(te, wload, res_dir):
    
    logging.info('# Create results folder for this execution')
    !mkdir {res_dir}
    
    logging.info('# Setup FTrace')
    te.ftrace.start()

    #logging.info('## Start energy sampling')
    #te.emeter.reset()

    logging.info('### Start RTApp execution')
    wload.run(out_dir=res_dir)

    #logging.info('## Read energy consumption: %s/energy.json', res_dir)
    #nrg_report = te.emeter.report(out_dir=res_dir)

    logging.info('# Stop FTrace')
    te.ftrace.stop()

    trace_file = os.path.join(res_dir, 'trace.dat')
    logging.info('# Save FTrace: %s', trace_file)
    te.ftrace.get_trace(trace_file)

    logging.info('# Save platform description: %s/platform.json', res_dir)
    plt, plt_file = te.platform_dump(res_dir)
    
    logging.info('# Report collected data:')
    logging.info('   %s', res_dir)
    !ls -la {res_dir}
    
    return plt, plt_file, trace_file

# Single task RTApp workload (1%)

In [9]:
rtapp_name = 'example_1pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p1': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 1,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-23 15:15:40,996 INFO    : Workload     : Setup new workload example_1pct
2018-07-23 15:15:41,099 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-23 15:15:41,170 INFO    : Workload     : Workload duration defined by longest task
2018-07-23 15:15:41,172 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-23 15:15:41,174 INFO    : Workload     : ------------------------
2018-07-23 15:15:41,177 INFO    : Workload     : task [task_p1], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-23 15:15:41,179 INFO    : Workload     :  | loops count: 1
2018-07-23 15:15:41,182 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-23 15:15:41,183 INFO    : Workload     : |  period    10000 [us], duty_cycle   1 %
2018-07-23 15:15:41,185 INFO    : Workload     : |  run_time    100 [us], sleep_time   9900 [us]
2018-07-23 15:15:41,187 DEBUG   : ssh          : /usr/bin/scp -r   example_1pct_00.json "ro

total 6876
drwxrwxr-x 2 leoy leoy    4096 Jul 23 15:16 .
drwxrwxr-x 3 leoy leoy    4096 Jul 23 15:15 ..
-rw-r--r-- 1 leoy leoy     586 Jul 23 15:15 example_1pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 23 15:15 output.log
-rw-rw-r-- 1 leoy leoy    1316 Jul 23 15:16 platform.json
-rw-r--r-- 1 leoy leoy  124160 Jul 23 15:15 rt-app-task_p10-0.log
-rw-r--r-- 1 leoy leoy  124160 Jul 23 15:15 rt-app-task_p1-0.log
-rw-r--r-- 1 leoy leoy  124160 Jul 23 15:15 rt-app-task_p15-0.log
-rw-rw-r-- 1 leoy leoy 6639616 Jul 23 15:16 trace.dat


,cpu,good_est,over_est,under_est
0,0,82,0,10
1,0,13,56,69
2,0,691,136,0
0,1,67,0,6
1,1,7,20,36
2,1,58,38,0
0,2,27,0,33
1,2,8,21,65
2,2,70,24,0
0,3,8,0,28


# Single task RTApp workload (3%)

In [10]:
rtapp_name = 'example_3pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p3': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 3,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-23 15:16:03,631 INFO    : Workload     : Setup new workload example_3pct
2018-07-23 15:16:03,739 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-23 15:16:03,811 INFO    : Workload     : Workload duration defined by longest task
2018-07-23 15:16:03,812 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-23 15:16:03,814 INFO    : Workload     : ------------------------
2018-07-23 15:16:03,816 INFO    : Workload     : task [task_p3], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-23 15:16:03,818 INFO    : Workload     :  | loops count: 1
2018-07-23 15:16:03,820 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-23 15:16:03,822 INFO    : Workload     : |  period    10000 [us], duty_cycle   3 %
2018-07-23 15:16:03,823 INFO    : Workload     : |  run_time    300 [us], sleep_time   9700 [us]
2018-07-23 15:16:03,825 DEBUG   : ssh          : /usr/bin/scp -r   example_3pct_00.json "ro

total 6576
drwxrwxr-x 2 leoy leoy    4096 Jul 23 15:16 .
drwxrwxr-x 4 leoy leoy    4096 Jul 23 15:16 ..
-rw-r--r-- 1 leoy leoy     586 Jul 23 15:16 example_3pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 23 15:16 output.log
-rw-rw-r-- 1 leoy leoy    1316 Jul 23 15:16 platform.json
-rw-r--r-- 1 leoy leoy  124160 Jul 23 15:16 rt-app-task_p3-0.log
-rw-rw-r-- 1 leoy leoy 6586368 Jul 23 15:16 trace.dat


,cpu,good_est,over_est,under_est
0,0,147,0,18
1,0,18,48,110
2,0,515,146,0
0,1,13,0,3
1,1,18,8,47
2,1,78,30,0
0,2,7,0,4
1,2,8,7,59
2,2,84,15,0
0,3,193,0,8


# Single task RTApp workload (5%)

In [11]:
rtapp_name = 'example_5pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p5': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 5,  # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-23 15:16:25,994 INFO    : Workload     : Setup new workload example_5pct
2018-07-23 15:16:26,096 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-23 15:16:26,169 INFO    : Workload     : Workload duration defined by longest task
2018-07-23 15:16:26,170 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-23 15:16:26,173 INFO    : Workload     : ------------------------
2018-07-23 15:16:26,174 INFO    : Workload     : task [task_p5], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-23 15:16:26,176 INFO    : Workload     :  | loops count: 1
2018-07-23 15:16:26,178 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-23 15:16:26,180 INFO    : Workload     : |  period    10000 [us], duty_cycle   5 %
2018-07-23 15:16:26,181 INFO    : Workload     : |  run_time    500 [us], sleep_time   9500 [us]
2018-07-23 15:16:26,183 DEBUG   : ssh          : /usr/bin/scp -r   example_5pct_00.json "ro

total 6560
drwxrwxr-x 2 leoy leoy    4096 Jul 23 15:16 .
drwxrwxr-x 5 leoy leoy    4096 Jul 23 15:16 ..
-rw-r--r-- 1 leoy leoy     586 Jul 23 15:16 example_5pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 23 15:16 output.log
-rw-rw-r-- 1 leoy leoy    1316 Jul 23 15:16 platform.json
-rw-r--r-- 1 leoy leoy  124160 Jul 23 15:16 rt-app-task_p5-0.log
-rw-rw-r-- 1 leoy leoy 6569984 Jul 23 15:16 trace.dat


,cpu,good_est,over_est,under_est
0,0,37,0,12
1,0,16,53,71
2,0,548,176,0
0,1,14,0,9
1,1,14,12,61
2,1,57,9,0
0,2,14,0,8
1,2,10,22,43
2,2,64,24,0
0,3,25,0,19


# Single task RTApp workload (10%)

In [12]:
rtapp_name = 'example_10pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p10': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 10, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-23 15:16:47,289 INFO    : Workload     : Setup new workload example_10pct
2018-07-23 15:16:47,391 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-23 15:16:47,463 INFO    : Workload     : Workload duration defined by longest task
2018-07-23 15:16:47,465 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-23 15:16:47,467 INFO    : Workload     : ------------------------
2018-07-23 15:16:47,469 INFO    : Workload     : task [task_p10], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-23 15:16:47,471 INFO    : Workload     :  | loops count: 1
2018-07-23 15:16:47,473 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-23 15:16:47,474 INFO    : Workload     : |  period    10000 [us], duty_cycle  10 %
2018-07-23 15:16:47,476 INFO    : Workload     : |  run_time   1000 [us], sleep_time   9000 [us]
2018-07-23 15:16:47,478 DEBUG   : ssh          : /usr/bin/scp -r   example_10pct_00.json 

total 6552
drwxrwxr-x 2 leoy leoy    4096 Jul 23 15:17 .
drwxrwxr-x 6 leoy leoy    4096 Jul 23 15:16 ..
-rw-r--r-- 1 leoy leoy     589 Jul 23 15:17 example_10pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 23 15:17 output.log
-rw-rw-r-- 1 leoy leoy    1316 Jul 23 15:17 platform.json
-rw-r--r-- 1 leoy leoy  124160 Jul 23 15:17 rt-app-task_p10-0.log
-rw-rw-r-- 1 leoy leoy 6561792 Jul 23 15:17 trace.dat


,cpu,good_est,over_est,under_est
0,0,46,0,68
1,0,16,47,78
2,0,1078,111,0
0,1,9,0,1
1,1,6,3,62
2,1,110,14,0
0,2,5,0,24
1,2,11,7,50
2,2,70,16,0
0,3,12,0,6


# Single task RTApp workload (15%)

In [13]:
rtapp_name = 'example_15pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p15': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 15, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-23 15:17:09,651 INFO    : Workload     : Setup new workload example_15pct
2018-07-23 15:17:09,753 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-23 15:17:09,820 INFO    : Workload     : Workload duration defined by longest task
2018-07-23 15:17:09,822 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-23 15:17:09,824 INFO    : Workload     : ------------------------
2018-07-23 15:17:09,825 INFO    : Workload     : task [task_p15], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-23 15:17:09,827 INFO    : Workload     :  | loops count: 1
2018-07-23 15:17:09,829 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-23 15:17:09,830 INFO    : Workload     : |  period    10000 [us], duty_cycle  15 %
2018-07-23 15:17:09,832 INFO    : Workload     : |  run_time   1500 [us], sleep_time   8500 [us]
2018-07-23 15:17:09,834 DEBUG   : ssh          : /usr/bin/scp -r   example_15pct_00.json 

total 6624
drwxrwxr-x 2 leoy leoy    4096 Jul 23 15:17 .
drwxrwxr-x 7 leoy leoy    4096 Jul 23 15:17 ..
-rw-r--r-- 1 leoy leoy     589 Jul 23 15:17 example_15pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 23 15:17 output.log
-rw-rw-r-- 1 leoy leoy    1316 Jul 23 15:17 platform.json
-rw-r--r-- 1 leoy leoy  124160 Jul 23 15:17 rt-app-task_p15-0.log
-rw-rw-r-- 1 leoy leoy 6635520 Jul 23 15:17 trace.dat


,cpu,good_est,over_est,under_est
0,0,57,0,8
1,0,24,46,89
2,0,741,206,0
0,1,30,0,5
1,1,18,8,67
2,1,55,13,0
0,2,29,0,13
1,2,12,18,67
2,2,75,32,0
0,3,18,0,7


# Single task RTApp workload (20%)

In [14]:
rtapp_name = 'example_20pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p20': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 20, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-23 15:17:31,641 INFO    : Workload     : Setup new workload example_20pct
2018-07-23 15:17:31,745 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-23 15:17:31,817 INFO    : Workload     : Workload duration defined by longest task
2018-07-23 15:17:31,818 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-23 15:17:31,821 INFO    : Workload     : ------------------------
2018-07-23 15:17:31,822 INFO    : Workload     : task [task_p20], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-23 15:17:31,824 INFO    : Workload     :  | loops count: 1
2018-07-23 15:17:31,827 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-23 15:17:31,829 INFO    : Workload     : |  period    10000 [us], duty_cycle  20 %
2018-07-23 15:17:31,830 INFO    : Workload     : |  run_time   2000 [us], sleep_time   8000 [us]
2018-07-23 15:17:31,832 DEBUG   : ssh          : /usr/bin/scp -r   example_20pct_00.json 

total 6716
drwxrwxr-x 2 leoy leoy    4096 Jul 23 15:17 .
drwxrwxr-x 8 leoy leoy    4096 Jul 23 15:17 ..
-rw-r--r-- 1 leoy leoy     589 Jul 23 15:17 example_20pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 23 15:17 output.log
-rw-rw-r-- 1 leoy leoy    1316 Jul 23 15:17 platform.json
-rw-r--r-- 1 leoy leoy  124160 Jul 23 15:17 rt-app-task_p20-0.log
-rw-rw-r-- 1 leoy leoy 6729728 Jul 23 15:17 trace.dat


,cpu,good_est,over_est,under_est
0,0,267,0,149
1,0,61,44,182
2,0,852,207,0
0,1,135,0,215
1,1,133,18,136
2,1,491,148,0
0,2,14,0,20
1,2,15,17,53
2,2,47,21,0
0,3,46,0,19


# Single task RTApp workload (25%)

In [15]:
rtapp_name = 'example_25pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p25': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 25, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-23 15:17:54,445 INFO    : Workload     : Setup new workload example_25pct
2018-07-23 15:17:54,547 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-23 15:17:54,619 INFO    : Workload     : Workload duration defined by longest task
2018-07-23 15:17:54,621 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-23 15:17:54,624 INFO    : Workload     : ------------------------
2018-07-23 15:17:54,625 INFO    : Workload     : task [task_p25], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-23 15:17:54,627 INFO    : Workload     :  | loops count: 1
2018-07-23 15:17:54,629 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-23 15:17:54,630 INFO    : Workload     : |  period    10000 [us], duty_cycle  25 %
2018-07-23 15:17:54,632 INFO    : Workload     : |  run_time   2500 [us], sleep_time   7500 [us]
2018-07-23 15:17:54,633 DEBUG   : ssh          : /usr/bin/scp -r   example_25pct_00.json 

total 6656
drwxrwxr-x 2 leoy leoy    4096 Jul 23 15:18 .
drwxrwxr-x 9 leoy leoy    4096 Jul 23 15:17 ..
-rw-r--r-- 1 leoy leoy     589 Jul 23 15:18 example_25pct_00.json
-rw-rw-r-- 1 leoy leoy     175 Jul 23 15:18 output.log
-rw-rw-r-- 1 leoy leoy    1316 Jul 23 15:18 platform.json
-rw-r--r-- 1 leoy leoy  124160 Jul 23 15:18 rt-app-task_p25-0.log
-rw-rw-r-- 1 leoy leoy 6668288 Jul 23 15:18 trace.dat


,cpu,good_est,over_est,under_est
0,0,33,0,8
1,0,20,41,66
2,0,730,129,0
0,1,16,0,10
1,1,8,16,43
2,1,52,14,0
0,2,25,0,16
1,2,7,20,63
2,2,54,18,0
0,3,14,0,5


# Single task RTApp workload (30%)

In [16]:
rtapp_name = 'example_30pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p30': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 30, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-23 15:18:16,588 INFO    : Workload     : Setup new workload example_30pct
2018-07-23 15:18:16,699 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-23 15:18:16,769 INFO    : Workload     : Workload duration defined by longest task
2018-07-23 15:18:16,772 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-23 15:18:16,773 INFO    : Workload     : ------------------------
2018-07-23 15:18:16,776 INFO    : Workload     : task [task_p30], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-23 15:18:16,778 INFO    : Workload     :  | loops count: 1
2018-07-23 15:18:16,779 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-23 15:18:16,781 INFO    : Workload     : |  period    10000 [us], duty_cycle  30 %
2018-07-23 15:18:16,783 INFO    : Workload     : |  run_time   3000 [us], sleep_time   7000 [us]
2018-07-23 15:18:16,785 DEBUG   : ssh          : /usr/bin/scp -r   example_30pct_00.json 

total 6660
drwxrwxr-x  2 leoy leoy    4096 Jul 23 15:18 .
drwxrwxr-x 10 leoy leoy    4096 Jul 23 15:18 ..
-rw-r--r--  1 leoy leoy     589 Jul 23 15:18 example_30pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 23 15:18 output.log
-rw-rw-r--  1 leoy leoy    1316 Jul 23 15:18 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 23 15:18 rt-app-task_p30-0.log
-rw-rw-r--  1 leoy leoy 6672384 Jul 23 15:18 trace.dat


,cpu,good_est,over_est,under_est
0,0,37,0,10
1,0,13,20,47
2,0,571,202,0
0,1,12,0,5
1,1,9,6,65
2,1,56,16,0
0,2,543,0,18
1,2,220,11,220
2,2,801,104,0
0,3,15,0,16


# Single task RTApp workload (35%)

In [17]:
rtapp_name = 'example_35pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p35': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 35, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-23 15:18:38,712 INFO    : Workload     : Setup new workload example_35pct
2018-07-23 15:18:38,817 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-23 15:18:38,883 INFO    : Workload     : Workload duration defined by longest task
2018-07-23 15:18:38,884 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-23 15:18:38,886 INFO    : Workload     : ------------------------
2018-07-23 15:18:38,887 INFO    : Workload     : task [task_p35], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-23 15:18:38,889 INFO    : Workload     :  | loops count: 1
2018-07-23 15:18:38,890 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-23 15:18:38,892 INFO    : Workload     : |  period    10000 [us], duty_cycle  35 %
2018-07-23 15:18:38,893 INFO    : Workload     : |  run_time   3500 [us], sleep_time   6500 [us]
2018-07-23 15:18:38,894 DEBUG   : ssh          : /usr/bin/scp -r   example_35pct_00.json 

total 6596
drwxrwxr-x  2 leoy leoy    4096 Jul 23 15:18 .
drwxrwxr-x 11 leoy leoy    4096 Jul 23 15:18 ..
-rw-r--r--  1 leoy leoy     589 Jul 23 15:18 example_35pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 23 15:18 output.log
-rw-rw-r--  1 leoy leoy    1316 Jul 23 15:18 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 23 15:18 rt-app-task_p35-0.log
-rw-rw-r--  1 leoy leoy 6606848 Jul 23 15:18 trace.dat


,cpu,good_est,over_est,under_est
0,0,28,0,16
1,0,20,39,81
2,0,437,73,0
0,1,18,0,3
1,1,8,15,65
2,1,55,26,0
0,2,14,0,11
1,2,22,16,72
2,2,55,9,0
0,3,13,0,4


# Single task RTApp workload (40%)

In [18]:
rtapp_name = 'example_40pct'
rtapp = RTA(target, rtapp_name, calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(

    # 1. generate a "profile based" set of tasks
    kind = 'profile',
    
    # 2. define the "profile" of each task
    params = {
        
        # 3. PERIODIC task with
        'task_p40': Periodic (
            period_ms      = 10, # period [ms]
            duty_cycle_pct = 40, # duty cycle [%]
            duration_s     = 10, # duration [s]
            delay_s        = 0,  # start after that delay [s]
            sched          = {   # run as a low-priority SCHED_OTHER task
                'policy'   : 'OTHER',
                'priotity' : 120,
            },
        ).get(),

    },
);

res_dir = os.path.join(te.res_dir, rtapp_name)
plt, plt_file, trace_file = execute(te, rtapp, res_dir)

prev_trace = Trace(hikey620_platform, trace_file, events=hikey620_conf['ftrace']['events'], normalize_time=True)
prev_ia = prev_trace.analysis.idle
df = dfAllCPUIdleStateEstimationStatistics(prev_ia, [ 1, 1000, 2700, 10000000000 ])
df

2018-07-23 15:19:00,882 INFO    : Workload     : Setup new workload example_40pct
2018-07-23 15:19:00,985 DEBUG   : ssh          : (mkdir -p /root/devlib-target); __devlib_ec=$?; echo; echo $__devlib_ec
2018-07-23 15:19:01,056 INFO    : Workload     : Workload duration defined by longest task
2018-07-23 15:19:01,058 INFO    : Workload     : Default policy: SCHED_OTHER
2018-07-23 15:19:01,060 INFO    : Workload     : ------------------------
2018-07-23 15:19:01,062 INFO    : Workload     : task [task_p40], sched: {'policy': 'OTHER', 'priotity': 120}
2018-07-23 15:19:01,064 INFO    : Workload     :  | loops count: 1
2018-07-23 15:19:01,065 INFO    : Workload     : + phase_000001: duration 10.000000 [s] (1000 loops)
2018-07-23 15:19:01,067 INFO    : Workload     : |  period    10000 [us], duty_cycle  40 %
2018-07-23 15:19:01,068 INFO    : Workload     : |  run_time   4000 [us], sleep_time   6000 [us]
2018-07-23 15:19:01,070 DEBUG   : ssh          : /usr/bin/scp -r   example_40pct_00.json 

total 6696
drwxrwxr-x  2 leoy leoy    4096 Jul 23 15:19 .
drwxrwxr-x 12 leoy leoy    4096 Jul 23 15:19 ..
-rw-r--r--  1 leoy leoy     589 Jul 23 15:19 example_40pct_00.json
-rw-rw-r--  1 leoy leoy     175 Jul 23 15:19 output.log
-rw-rw-r--  1 leoy leoy    1316 Jul 23 15:19 platform.json
-rw-r--r--  1 leoy leoy  124160 Jul 23 15:19 rt-app-task_p40-0.log
-rw-rw-r--  1 leoy leoy 6709248 Jul 23 15:19 trace.dat


,cpu,good_est,over_est,under_est
0,0,48,0,14
1,0,16,39,75
2,0,682,81,0
0,1,426,0,221
1,1,717,140,564
2,1,141,40,0
0,2,24,0,8
1,2,9,16,53
2,2,67,22,0
0,3,10,0,7
